In [1]:
import pandas as pd

In [2]:
binsize = 400
hashid = 'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89'

df_stations = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

df_stations = df_stations[df_stations['hash'] == hashid]

my_data_filename = 'data/C2A2_data/BinnedCsvs_d{}/{}.csv'.format(binsize, hashid)
df_my_data = pd.read_csv(my_data_filename, parse_dates=[0])

df = pd.merge(df_stations[['ID']], df_my_data, how='left', on='ID', sort=False)
df['Data_Value'] = df['Data_Value'].apply(lambda x: x/10)

In [3]:
df = df[((df['Date'] >= '2005-01-01') & (df['Date'] <= '2005-01-02')) | ((df['Date'] >= '2006-01-01') & (df['Date'] <= '2006-01-02'))]
# df = df[(df['Date'] == '2005-01-01') | (df['Date'] == '2006-01-01')]

In [4]:
del df['ID']

In [5]:
# df.reset_index(inplace=True)
# df.drop('index', axis = 1, inplace=True)
# df = df.set_index(['Date', 'Element'])
# df.sort_values(['Date', 'Data_Value'])

In [6]:
df.drop_duplicates(inplace=True)

In [7]:
df.loc[df.groupby('Date').Data_Value.idxmin()]

,Date,Element,Data_Value
158310,2005-01-01,TMIN,-5.6
68651,2005-01-02,TMIN,-5.6
77331,2006-01-01,TMIN,-3.9
78785,2006-01-02,TMIN,-1.1


In [8]:
min_df = df[df['Data_Value'] == df.groupby('Date')['Data_Value'].transform('min')]
del min_df['Element']
min_df.columns = ['date', 'min']

In [9]:
max_df = df[df['Data_Value'] == df.groupby('Date')['Data_Value'].transform('max')]
del max_df['Element']
max_df.columns = ['date', 'max']

In [10]:
merged_df = pd.merge(min_df, max_df, how='left', sort=False)

In [11]:
merged_df.sort_values('date')

,date,min,max
3,2005-01-01,-5.6,15.6
0,2005-01-02,-5.6,13.9
1,2006-01-01,-3.9,3.9
2,2006-01-02,-1.1,4.4


In [12]:
merged_df['date'] = pd.to_datetime(merged_df['date'], format='%Y-%m-%d')

In [13]:
merged_df = merged_df.drop(merged_df[~((merged_df.date.dt.month == 2) & (merged_df.date.dt.day == 29))].index)

In [14]:
merged_df['date'].dtype

dtype('<M8[ns]')